# A preliminary pipeline to collect features

In [1]:
import jusipy
import pandas as pd

In [2]:
# Load datasets
LM        = jusipy.deals.LandMatrix()
A_country = jusipy.country_features.All()
A_latlong = jusipy.latlong_features.All()
GLCF_8km  = A_latlong.datasets[2]         # extract from A_latlong, just to not load this one twice...

Loading GLCF(8km)                                                                                                     reas)                                                                                                      

In [3]:
# Load GIS tools
GC = jusipy.GIS.GoogleCode(cache=jusipy.utils.FireDict(sub='googlecode'))
CC = jusipy.GIS.CountryCode()

## Select the points, and get the relevant data

### Positive points from LandMatrix

In [ ]:
positive_points = LM.M[['lat', 'long', 'target_country_iso3', 'year']]
positive_points = positive_points.rename(columns={'target_country_iso3' : 'country'})
positive_points['positive'] = 1
positive_points['random']   = 0

### Random points selected from around the earth

In [ ]:
#random_points = pd.DataFrame(jusipy.GIS.sample.random_latlong(land=True, glcf=GLCF_8km, size=10000),
#                             columns=['lat', 'long'])

# For each random point, get the iso3 country ID
rp_iso3 = random_points.apply(lambda r: CC[GC.latlong(r.lat, r.long).get('country_political', 'XXXXXXX')].iso3,
                              axis=1)
random_points['country'] = rp_iso3
random_points['year'] = ['newest'] * rp_iso3
random_points['positive'] = 0
random_points['random']   = 1

### Combine the two stacks of data

In [ ]:
all_points = pd.zcat([positive_points, random_points])
all_points

## Visualize the Spatial data

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(all_points.long, all_points.lat, c=all_points.positive, s=2, zorder=1)

## Collect features for each point

In [ ]:
country_features = jusipy.country_features.get(all_points, A_country)
latlong_features = jusipy.latlong_features.get(all_points, A_latlong)
all_features = country_features.join(latlong_features)

## Store the output

In [ ]:
all_points.to_pickle('data/all_labels.pkl')
all_features.to_pickle('data/all_features.pkl')